### Imports

In [2]:
import numpy as np
import scipy.constants as const
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt

### Constants

In [3]:
bohr_factor = const.value('Bohr radius')
# Conversion factor angstrom to bohr
angstrom_to_bohr = const.angstrom / bohr_factor
hartree_to_kcal = 627.509
kJ_to_kcal = 1 / 4.184 

# Functions

In [ ]:
def param_asig_abc(vec_mol):

    # Dictionary here to get the values depending of the type of atom
    # Some Atoms doesn't have the correct values(not for quin, qith quin is all good) 
    force_field_params = {'H(1)': {'A': 278.37, 'B': 12680, 'C': 3.56},
                          'H(2)': {'A': 0.00, 'B': 361.3, 'C': 3.56},
                          'H(3)': {'A': 0.00, 'B': 764.9, 'C': 3.56},
                          'H(4)': {'A': 0.00, 'B': 1.02385747e+03, 'C': 3.56},  # Asumiendo el mismo valor para H(4) como H(3)
                          'C(4)': {'A': 978.36, 'B': 131571, 'C': 3.60},
                          'C(3)': {'A': 1701.73, 'B': 170356, 'C': 3.60},
                          'C(2)': {'A': 1435.09, 'B': 103235, 'C': 3.60},
                          'N(1)': {'A': 1407.57, 'B': 96349, 'C': 3.48},
                          'N(2)': {'A': 1398.15, 'B': 102369, 'C': 3.48},
                          'N(3)': {'A': 2376.55, 'B': 191935, 'C': 3.48},
                          'N(4)': {'A': 5629.82, 'B': 405341, 'C': 3.48},
                          'O(2)': {'A': 1285.87, 'B': 284262, 'C': 3.96},
                          'O(1)': {'A': 1260.73, 'B': 241042, 'C': 3.96},
                          'P':{'A': 283248.0316091423, 'B': 273570.9941372, 'C': 4.0775266},
                          'F':{'A': 283248.0316091423, 'B': 273570.9941372, 'C': 4.0775266}}
    
    force_field_params = {'H(1)': {'A': 2.78370000e+02, 'B': 6.33352353e+03, 'C': 3.29777550e+00},
                          'H(2)': {'A': 0.00, 'B': 361.3, 'C': 3.56},
                          'H(3)': {'A': 0.00, 'B': 764.9, 'C': 3.56},
                          'H(4)': {'A': 0.00, 'B': 1.02385747e+03, 'C': 3.56},  # Asumiendo el mismo valor para H(4) como H(3)
                          'C(4)': {'A': 9.78360000e+02, 'B': 1.00000000e+04, 'C': 3.60},
                          'C(3)': {'A': 1701.73, 'B': 170356, 'C': 3.60},
                          'C(2)': {'A': 1435.09, 'B': 103235, 'C': 3.60},
                          'N(1)': {'A': 1407.57, 'B': 96349, 'C': 3.48},
                          'N(2)': {'A': 1398.15, 'B': 102369, 'C': 3.48},
                          'N(3)': {'A': 2376.55, 'B': 191935, 'C': 3.48},
                          'N(4)': {'A': 1.71791554e+03, 'B': 1.66965018e+05, 'C': 3.37316262e+00},
                          'O(2)': {'A': 1285.87, 'B': 284262, 'C': 3.96},
                          'O(1)': {'A': 1260.73, 'B': 241042, 'C': 3.96},
                          'P':{'A': 283248.0316091423, 'B': 273570.9941372, 'C': 4.0775266},
                          'F':{'A': 283248.0316091423, 'B': 273570.9941372, 'C': 4.0775266}} 
    
    # A-> C , C-> B, B->A 
    

    vec_a = []
    vec_b = []
    vec_c = []

    for item in vec_mol:
        if item in force_field_params:
            atom_params = force_field_params[item]
            vec_a.append(atom_params['A'])
            vec_b.append(atom_params['B'])
            vec_c.append(atom_params['C'])
        else:
            # Manejar el caso de un átomo no definido, por ejemplo, agregando un valor predeterminado o lanzando un error
            print(f"Advertencia: {item} no está definido en los parámetros del campo de fuerza.")

    return np.array(vec_a), np.array(vec_b), np.array(vec_c)

def combination_abc(vec_1, vec_2):
    
    vec_1 = np.array(vec_1)
    vec_2 = np.array(vec_2)

    vec_1a, vec_1b, vec_1c = param_asig_abc(vec_1)
    vec_2a, vec_2b, vec_2c = param_asig_abc(vec_2)

    v_12a = (np.outer(vec_1a, vec_2a).flatten())**(0.5)
    v_12b = (np.outer(vec_1b, vec_2b).flatten())**(0.5)
    v_12c = ((0.5)*(vec_1c[:, np.newaxis] + vec_2c)).flatten()  

    return np.array(v_12a), np.array(v_12b), np.array(v_12c)


def space_distances_generation(base_molecule, configurations):
    space_distance_vec = []
    for w in range(len(configurations)):  # w is the distances vector
        space_distance = cdist(base_molecule, configurations[w], 'euclidean')
        space_distance = np.array(space_distance.flatten())
        space_distance_vec.append(space_distance)
    return np.array(space_distance_vec)


def vector_q1q2(vec_1, vec_2):
    v_q1q2 = np.outer(vec_1, vec_2).flatten()
    return np.array(v_q1q2)


def xyz_a_matriz_numpy(ruta_archivo_xyz):
    # Leer el contenido del archivo
    with open(ruta_archivo_xyz, 'r') as archivo:
        lineas = archivo.readlines()

    # Omitir la primera línea (contiene el número total de átomos) y otras no necesarias
    # Asumimos que las coordenadas comienzan desde la tercera línea
    lineas_coordenadas = lineas[2:]
    
    # Preparar una lista para almacenar las coordenadas
    coordenadas = []
    
    for linea in lineas_coordenadas:
        if linea.strip():  # Verificar que la línea no esté vacía
            partes = linea.split()
            if len(partes) == 4:  # Asegurar que la línea tenga el formato esperado (Elemento x y z)
                # Convertir las partes numéricas (x, y, z) a flotantes y añadir a la lista
                coordenadas.append([float(partes[1]), float(partes[2]), float(partes[3])])

    # Convertir la lista de coordenadas a una matriz NumPy
    matriz_coordenadas = np.array(coordenadas)

    return matriz_coordenadas


def matrix_d_abc_q1q2(base_molecule, configurations, vec_id_1, vec_id_2,
                      vec_q1, vec_q2):
    """ remember to follow the "naturality" of the matrixes, molecule 1 -->id1
  molecule2 vec_id2 and vec q2 etc, to have in the flatten the correct combi"""

    vec_d = space_distances_generation(base_molecule, configurations)
    vec_a, vec_b, vec_c = combination_abc(vec_id_1, vec_id_2)
    vec_q1q2 = vector_q1q2(vec_q1, vec_q2)

    combination_matrix = []
    for vector in vec_d:
        combination_matrix.append(np.vstack((vector, vec_a, vec_b, vec_c, vec_q1q2)))

    return combination_matrix  



def energy_williams(combination_matrix):
    combination_matrix = np.array(combination_matrix)
    energies = []
    for i in range(combination_matrix.shape[0]):
        sum = 0
        for j in range(combination_matrix.shape[2]):
            exp_term = (combination_matrix[i, 2, j])*(np.exp((-combination_matrix[i, 3, j])*(combination_matrix[i, 0, j])))
            dis_ind_term = combination_matrix[i, 1, j]/((combination_matrix[i, 0, j])**6)
            elst_term = (combination_matrix[i, 4, j])/(combination_matrix[i, 0, j]*angstrom_to_bohr)
            sum = sum + (exp_term*kJ_to_kcal) - (dis_ind_term*kJ_to_kcal) + (elst_term*hartree_to_kcal)
        energies.append(sum)
    return energies